# import Libraries 

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
from langchain.agents import Tool, initialize_agent, load_tools


In [2]:
from langchain_groq import ChatGroq
from IPython.display import Markdown, display

In [3]:
load_dotenv()
GROQ_API = os.getenv("GROQ_API_KEY")
model_name = 'gemma2-9b-it'


llm  =  ChatGroq(api_key=GROQ_API,
    model_name=model_name,
    temperature=0.0,
    
)

In [4]:
tools = load_tools(
    ['llm-math'],
    llm=llm
)

print(tools[0].name, tools[0].description)

Calculator Useful for when you need to answer questions about math.


In [8]:
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3 # to avoid high bills from the LLM
)


## ReAct framework = Reasoning and Action

## ReAct Methodology
The chatbot employs a step-by-step reasoning approach to interpret and respond to user prompts:
1. Thought: Analyze the prompt internally.
2. Action: Execute an action, such as calling a tool (SQL or image processing).
3. Observation: Incorporate the tool’s output into subsequent reasoning steps.
4. Answer: Provide the final response. 


In [9]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [10]:
query = "If James is currently 45 years old, how old will he be in 50 years? \
    If he has 4 kids and adopted 7 more, how many children does he have?"
result = agent.invoke(query)
print(result['output'])



> Entering new AgentExecutor chain...
Thought: To find James's age in 50 years, I need to add 50 to his current age. To find the total number of children, I need to add the number of biological children to the number of adopted children. 

Action: Calculator
Action Input: 45 + 50
Observation: Answer: 95
Thought:Thought: I now know James's age in 50 years. Now I need to add his biological children to his adopted children. 
Action: Calculator
Action Input: 4 + 7
Observation: Answer: 11
Thought:Thought: I now know the final answer
Final Answer: James will be 95 years old in 50 years. He has 11 children. 


> Finished chain.
James will be 95 years old in 50 years. He has 11 children.


In [12]:

#display(Markdown(f" ChatGPT ::: {llm.predict('what is 3.1^2.1')}"))
print(f" ChatGPT ::: {llm.predict('what is 3.1^2.1')}")





 ChatGPT ::: 3.1 raised to the power of 2.1 can be calculated using a calculator.  

Here's how to do it:

* **Input:** Enter "3.1" followed by "^" (the exponent symbol), then "2.1".
* **Calculate:** Press the equals button (=).

The result is approximately **10.38**. 


Let me know if you'd like to calculate any other exponents! 



In [13]:
result = agent.invoke('what is 3.1^2.1')
print(result['output'])



> Entering new AgentExecutor chain...
Thought: I can use the calculator to find the result of 3.1 raised to the power of 2.1. 
Action: Calculator
Action Input: 3.1 ** 2.1
Observation: Answer: 10.761171606099687
Thought:Thought: I now know the final answer
Final Answer: 10.761171606099687 


> Finished chain.
10.761171606099687


In [20]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [22]:
docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="search wikipedia"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="lookup a term in wikipedia"
    )
]


In [29]:
# initialize our agent
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=10
)

In [30]:
query = "What were Einstein's main beliefs?"
result = docstore_agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: I need to search for information about Albert Einstein's beliefs. 

Action: Search[Albert Einstein beliefs] 

Observation: Albert Einstein's religious views have been widely studied and often misunderstood. Albert Einstein stated "I believe in Spinoza's God". He did not believe in a personal God who concerns himself with fates and actions of human beings, a view which he described as naïve. He clarified, however, that, "I am not an atheist", preferring to call himself an agnostic, or a "religious nonbeliever." In other interviews, he stated that he thought that there is a "lawgiver" who sets the laws of the universe. Einstein also stated he did not believe in life after death, adding "one life is enough for me." He was closely involved in his lifetime with several humanist groups. Einstein rejected a conflict between science and religion, and held that cosmic religion was necessary for science.
Thought:Einstein's main beliefs were:

* **

In [31]:
print(docstore_agent.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

In [67]:
import os
from dotenv import find_dotenv, load_dotenv
import openai
from langchain.llms import OpenAI
from langchain.agents import Tool, initialize_agent, load_tools

from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [34]:
# memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [35]:
# Second Generic Tool
prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

In [36]:
llm_chain = prompt | llm

In [38]:
# Initialize the LLM Tool
llm_tool = Tool(
    name="Language Model",
    func=llm_chain.invoke,
    description="Use this tool for general queries and logic"
)
 
tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [39]:
tools.append(llm_tool) # adding the new tool to our tools list


In [44]:
# Conversational Agent
conversational_agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=4,
    memory=memory
)

In [45]:
query = "How old is a person born in 1917 in 2023"

In [46]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [48]:
result = conversational_agent(query)
print(result['output'])




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: A person born in 1917 would be 106 years old in 2023. 


> Finished chain.
A person born in 1917 would be 106 years old in 2023.


In [55]:
query_two = "How old would that person be if their age is multiplied by 100 and add resulte this 159*857?"

In [56]:
results = conversational_agent(query_two)
print(results['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 159 * 857
Observation: Answer: 136263
Thought:Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 10659 * 136263
Observation: Answer: 1452427317
Thought:Thought: Do I need to use a tool? No
AI: That person would be 1,452,427,317 years old. 


> Finished chain.
That person would be 1,452,427,317 years old.
